In [1]:
import pandas as pd
import numpy as np
from numpy import nan

#store as a dataframe
pd.set_option("display.max_colwidth", -1)
# df = pd.read_excel("Movement Report 1.xls",skiprows =12,header = [0,1,2])
# df = pd.read_excel("Movement Report 2.xls",skiprows =12,header = [0,1,2])
# df = pd.read_excel("Movement Report 3.xls",skiprows =12,header = [0,1,2])
# df = pd.read_excel("Movement Report 4.xls",skiprows =12,header = [0,1,2])
# df = pd.read_excel("Movement Report 5.xls",skiprows =12,header = [0,1,2])
# df = pd.read_excel("Movement Report 6.xls",skiprows =12,header = [0,1,2])
# df = pd.read_excel("Movement Report 7.xls",skiprows =12,header = [0,1,2])
# df = pd.read_excel("Movement Report 8.xls",skiprows =12,header = [0,1,2])
# df = pd.read_excel("Movement Report 9.xls",skiprows =12,header = [0,1,2])
# df = pd.read_excel("Movement Report 10.xls",skiprows =12,header = [0,1,2])
# df = pd.read_excel("Movement Report 11.xls",skiprows =12,header = [0,1,2])
# df = pd.read_excel("Movement Report 12.xls",skiprows =12,header = [0,1,2])
df = pd.read_excel("Movement Report 13.xls",skiprows =12,header = [0,1,2])
original = df
df.head()

C:\Users\yjkwak.2018\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


FileNotFoundError: [Errno 2] No such file or directory: 'Movement Report 1.xls'

In [ ]:
#clean up table headers & standardize to small letters 
ans = []
for title in df.columns:
    name = ""
    for element in title:
        if name == "":
            name += element
        elif "Unnamed" not in element:
            name += " " +element 
    ans.append(name.lower())
df.columns = ans
df.head()

In [ ]:
names = df.columns.values
names_list = ["drop","speeding count","braking","accel","cornering"]
for i in range(1,len(names_list)+1):
    word = names_list[-i]
    names[-i] = word
df.columns = names
del df["drop"]
df

In [ ]:
#delete useless columns 
#edit useful list accordingly 
useful = ["date","registration","start trip time >10km/hr","start trip location","end trip time","end trip location","distance traveled","working duration","geofence visited in trip","speeding count","braking","accel","cornering"]

for title in df.columns: 
    if title not in useful:
        while title in df.columns: #may have duplicated columns that have already been deleted
            del df[title]
    #print(df.columns)
df.head(5)

In [ ]:
for row in df.iterrows():
    print(row)
    for i in range(0,len(df.colums))

In [ ]:
#delete row if all the columns in the row is NaN 
row_no = 0 
rows_dropped = []
deleted = []
for row in df.iterrows():
#     print(row)
#     print(row[1])
    count = 0
#     print("entering row now")
    for i in range(0,len(df.columns)):
        if  pd.isnull(row[1][i])== True:
#             print(row[1])
            count+= 1
        if row[1][i] in ("Totals","Registration","time",">10km/hr","Speeding count") :
            rows_dropped.append(row_no)
            deleted.append(row)
            df = df.drop([row_no])
    if count == len(df.columns):
#         print("true")
        rows_dropped.append(row_no)
        deleted.append(row)
        df = df.drop([row_no])
#         print("drop successful")
#     print("end the process -----------------------")
    row_no+=1
# print(rows_dropped) # row number of deleted rows
#deleted >> collection of deleted rows to double chk 
df.reset_index(drop=True, inplace=True)
df.head()

In [ ]:
num = 0 
emp = 0 
max = len(df)
while num <= max:
    if num == max: 
#         print(num)
#         print(emp)
        for i in range (emp,0,-1):
            string3 = str(df.loc[num-i][3])
            string5 = str(df.loc[num-i][5])
            if string3 != "nan":
                df.loc[num-emp-1][3] = str(df.loc[num-emp-1][3]) + string3
            if string5 != "nan":
                df.loc[num-emp-1][5] = str(df.loc[num-emp-1][5]) + string5
            df = df.drop([num-i])
        emp = 0
        num+=1
    elif pd.isnull(df.loc[num][0])== True:
#         print(num,1)
        emp+=1
        num+=1
    elif emp > 0:
        for i in range (emp,0,-1):
            string3 = str(df.loc[num-i][3])
            string5 = str(df.loc[num-i][5])
            if string3 != "nan":
                df.loc[num-emp-1][3] = str(df.loc[num-emp-1][3]) + string3
            if string5 != "nan":
                df.loc[num-emp-1][5] = str(df.loc[num-emp-1][5]) + string5
            df = df.drop([num-i])
#         print(num,2)
        emp = 0
        num+=1
    else:
#         print(num,3)
        num +=1 
        
print("the end")

In [ ]:
df.reset_index(drop=True, inplace=True)
df.head()

In [ ]:
# compilation = pd.DataFrame()
compilation = compilation.append(df)
print(len(compilation))
compilation

In [ ]:
compilation.reset_index(drop=True, inplace=True)
compilation.head()
print(len(compilation))
compilation[compilation.duplicated()]

In [ ]:
compilation = compilation.drop_duplicates(keep='last')
print(len(compilation))

In [ ]:
compilation.to_csv('mov.csv')
